In [6]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import os
import random

# --- CONFIGURATION ---
# 🚨 IMPORTANT: Replace this placeholder with the ACTUAL PATH to your dataset's 'train' folder.
# This folder must contain the class subdirectories (e.g., Tomato___Healthy, Tomato___Late_blight, etc.)
DATA_DIR = 'tomato/train'  
MODEL_FILE_NAME = 'tomato_disease_classifier.h5'

IMAGE_SIZE = (128, 128)  # Standard input size for the CNN
BATCH_SIZE = 32

# --- STEP 1: DATA PREPARATION (Keras Generators) ---

print("1. Setting up Data Generators...")

# Initialize the generator with normalization and data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,             # Normalize pixel values (0 to 1)
    validation_split=0.2,       # Use 20% of data for validation

    # Data Augmentation (Crucial for better model generalization)
    rotation_range=20,          
    horizontal_flip=True,       
    zoom_range=0.1              
)

# Training Data Generator: Loads the training subset
try:
    train_generator = datagen.flow_from_directory(
        DATA_DIR,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training',
        shuffle=True
    )

    # Validation Data Generator: Loads the validation subset
    validation_generator = datagen.flow_from_directory(
        DATA_DIR,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation',
        shuffle=False
    )
    
    class_names = list(train_generator.class_indices.keys())
    NUM_CLASSES = len(class_names)

    if train_generator.n == 0:
         raise ValueError(f"Found 0 images in the training subset. Please verify the DATA_DIR path: {DATA_DIR}")
    
    print(f"Detected {NUM_CLASSES} Classes: {class_names}")
    print(f"Total Training Images: {train_generator.n}")
    print(f"Total Validation Images: {validation_generator.n}")

except Exception as e:
    print(f"\n--- FATAL ERROR IN DATA LOADING ---")
    print("Check the folder structure and path again. Details:")
    print(e)
    exit() 

# --- STEP 2: BUILD AND TRAIN THE CNN MODEL ---

def create_cnn_model(input_shape, num_classes):
    """Creates a simple Convolutional Neural Network model."""
    model = Sequential([
        # Feature Extraction Layers
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        
        # Classification Head
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5), # Regularization
        Dense(num_classes, activation='softmax') # Output layer
    ])
    
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

model = create_cnn_model(IMAGE_SIZE + (3,), NUM_CLASSES)

print("\n2. Starting Model Training (5 Epochs)...")
# Note: You can increase the number of epochs for better accuracy if you have time.
history = model.fit(
    train_generator,
    epochs=5, 
    validation_data=validation_generator
)

print(f"\nTraining Complete. Model saved as '{MODEL_FILE_NAME}'")
model.save(MODEL_FILE_NAME)

# --- STEP 3: PREDICTION FUNCTION ---

def predict_leaf_disease(image_path, model, class_names, target_size):
    """Predicts the disease class of a new image."""
    
    # 1. Load and Preprocess using OpenCV
    img_cv = cv2.imread(image_path)
    if img_cv is None:
        return f"ERROR: Image not found at {image_path}.", None
        
    img_rgb = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB) # Convert BGR (OpenCV default) to RGB
    img_resized = cv2.resize(img_rgb, target_size)
    
    img_array = np.array(img_resized) / 255.0  # Normalize to 0-1
    img_array = np.expand_dims(img_array, axis=0) # Add batch dimension

    # 2. Predict
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    
    # 3. Interpret Result
    predicted_class_index = np.argmax(score)
    confidence = np.max(score) * 100
    predicted_class = class_names[predicted_class_index]
    
    result = (
        f"Prediction: **{predicted_class.replace('___', ' ')}**\n"
        f"Confidence: {confidence:.2f}%"
    )
    
    return result, score

# --- Example Prediction (Optional: Uncomment to test) ---
# # To test: Get a path to any image in one of your class folders and paste it here.
# # test_image_path = '/path/to/your/Kaggle/Tomato_leaf_disease_detection/train/Tomato___Healthy/0001.jpg'
# 
# # try:
# #     prediction_result, scores = predict_leaf_disease(test_image_path, model, class_names, IMAGE_SIZE)
# #     print(f"\n--- SAMPLE PREDICTION ---\n{prediction_result}")
# # except NameError:
# #     print("Prediction failed. Ensure 'test_image_path' is defined and the model has finished training.")

1. Setting up Data Generators...
Found 8000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Detected 10 Classes: ['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']
Total Training Images: 8000
Total Validation Images: 2000

2. Starting Model Training (5 Epochs)...
Epoch 1/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 309s 1s/step - accuracy: 0.2626 - loss: 2.1345 - val_accuracy: 0.6170 - val_loss: 1.0704
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 322s 1s/step - accuracy: 0.5887 - loss: 1.1943 - val_accuracy: 0.7465 - val_loss: 0.7724
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 288s 1s/step - accuracy: 0.6666 - loss: 0.9800 - val_accuracy: 0.7970 - val_loss: 0.6501
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 278s 1s/step - accuracy: 0.7125 - loss: 


Training Complete. Model saved as 'tomato_disease_classifier.h5'


In [10]:
test_image_path = 'tomato/train/Tomato___Tomato_Yellow_Leaf_Curl_Virus/d01c276a-a70d-4981-930e-d78292219f61___UF.GRC_YLCV_Lab 02057.JPG'
try:
    prediction_result, scores = predict_leaf_disease(test_image_path, model, class_names, IMAGE_SIZE)
    print(f"\n--- SAMPLE PREDICTION ---\n{prediction_result}")
except NameError:
    print("Prediction failed. Ensure 'test_image_path' is defined and the model has finished training.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step

--- SAMPLE PREDICTION ---
Prediction: **Tomato Tomato_Yellow_Leaf_Curl_Virus**
Confidence: 23.11%


In [8]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import os
import time

# --- CONFIGURATION ---
MODEL_FILE_NAME = 'tomato_disease_classifier.h5' 
IMAGE_SIZE = (128, 128) # Must match the size used during training

# --- STEP 1: LOAD MODEL AND CLASS NAMES ---

try:
    print(f"Loading model from: {MODEL_FILE_NAME}...")
    model = load_model(MODEL_FILE_NAME)

    # 🚨 IMPORTANT: Define class names in the EXACT order they were learned by the generator.
    # If you trained on the original Kaggle dataset, use this list (or the list saved during training).
    class_names = [
        'Tomato___Bacterial_spot', 
        'Tomato___Early_blight',
        'Tomato___Healthy',
        'Tomato___Late_blight',
        'Tomato___Leaf_Mold',
        'Tomato___Septoria_leaf_spot',
        'Tomato___Spider_mites Two-spotted_spider_mite',
        'Tomato___Target_Spot',
        'Tomato___Tomato_mosaic_virus',
        'Tomato___Tomato_Yellow_Leaf_Curl_Virus'
    ]
    
except FileNotFoundError:
    print(f"\n--- ERROR: Model Not Found ---")
    print(f"Please ensure '{MODEL_FILE_NAME}' is in the current directory.")
    exit()
except Exception as e:
    print(f"\n--- ERROR: Could not load model or classes ---")
    print(f"Details: {e}")
    exit()

# --- STEP 2: CLASSIFICATION FUNCTION (Modified to accept a frame) ---

def classify_frame(frame, model, class_names, target_size):
    """Preprocesses a captured frame and makes a prediction."""
    
    # 1. Preprocessing
    # Convert BGR (OpenCV frame default) to RGB 
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) 
    
    # Resize to the model's required input size
    img_resized = cv2.resize(img_rgb, target_size)
    
    # Convert to NumPy array and normalize (0-1)
    img_array = np.array(img_resized) / 255.0  
    
    # Add a batch dimension: (1, 128, 128, 3)
    img_array = np.expand_dims(img_array, axis=0) 

    # 2. Make Prediction
    predictions = model.predict(img_array)
    probabilities = tf.nn.softmax(predictions[0]) 
    
    # Get the result
    predicted_index = np.argmax(probabilities)
    confidence = np.max(probabilities) * 100
    predicted_class = class_names[predicted_index]
    
    return predicted_class, confidence

# --- STEP 3: CAMERA CAPTURE LOOP ---

def run_live_classifier():
    # Initialize webcam (0 is usually the default camera)
    cap = cv2.VideoCapture(0) 
    
    if not cap.isOpened():
        print("\n--- ERROR ---: Cannot open camera. Check camera index or connection.")
        return

    print("\n--- LIVE CLASSIFIER READY ---")
    print("Press 'c' to Capture and Classify the current frame.")
    print("Press 'q' to Quit.")

    # Variables for displaying the last result
    last_prediction = "Awaiting Capture..."
    last_confidence = 0.0

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break

        # Display the prediction text on the live feed
        display_frame = frame.copy()
        text = f"Condition: {last_prediction} | Conf: {last_confidence:.2f}%"
        cv2.putText(display_frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(display_frame, "Press 'c' to CAPTURE", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

        cv2.imshow('Tomato Leaf Live Classifier', display_frame)

        # Handle key presses
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('q'):
            break
        
        elif key == ord('c'):
            print("\n[INFO] Capturing and classifying...")
            
            # Run the prediction
            predicted_class, confidence = classify_frame(frame, model, class_names, IMAGE_SIZE)
            
            # Update the displayed text variables
            last_prediction = predicted_class.replace('___', ' ')
            last_confidence = confidence

            # Log the result
            print(f"   -> Result: {last_prediction} ({confidence:.2f}%)")
            
            # Provide actionable guidance based on the result
            if "Healthy" in last_prediction:
                print("   -> Guidance: 🌱 KEEP. Plant is healthy.")
            elif "Virus" in last_prediction or "Late_blight" in last_prediction:
                print("   -> Guidance: ⚠️ REMOVE. Severe and highly contagious disease.")
            else:
                print("   -> Guidance: 🧐 TREAT. Moderate disease, requires treatment.")
            
            # Brief pause to visually emphasize the capture
            time.sleep(0.5) 

    # When everything is done, release the capture
    cap.release()
    cv2.destroyAllWindows()

# Execute the live classifier
run_live_classifier()

Loading model from: tomato_disease_classifier.h5...



--- LIVE CLASSIFIER READY ---
Press 'c' to Capture and Classify the current frame.
Press 'q' to Quit.

[INFO] Capturing and classifying...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 450ms/step
   -> Result: Tomato Early_blight (20.73%)
   -> Guidance: 🧐 TREAT. Moderate disease, requires treatment.
